In [ ]:
!pip install lime
!pip install interpret
!pip install eli5
!pip install shap

In [ ]:
!pip install scikit-learn==1.2.2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import IPython
import numpy as np
import eli5
from eli5 import show_prediction
import shap
import lime.lime_tabular
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

In [ ]:
credito = pd.read_csv('Credit.csv')

In [ ]:
credito.shape

In [ ]:
credito.head()

In [ ]:
previsores = credito.iloc[:, :-1].values
classe = credito.iloc[:, -1].values

In [ ]:
labelEncoder = LabelEncoder()

for i in range(previsores.shape[1]):
  if previsores[:, i].dtype == "object":
    previsores[:, i] = labelEncoder.fit_transform(previsores[:, i])

In [ ]:
classe = labelEncoder.fit_transform(classe)

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(previsores, classe, test_size=0.3)

In [ ]:
modelo = RandomForestClassifier(n_estimators = 1000)
modelo.fit(X_treino, y_treino)

In [ ]:
# LIME -> é agnóstico
expl = lime.lime_tabular.LimeTabularExplainer(X_treino, feature_names = list(credito)[0:20], class_names="class")
prever = lambda x: modelo.predict_proba(x).astype(float)

exp = expl.explain_instance(X_teste[0], prever, num_features=5)
exp.show_in_notebook(show_all=True)

In [ ]:
# ELI5 -> pode ser tanto global quanto local
eli5.show_weights(modelo, feature_names = list(credito)[0:20]) # FORMA GLOBAL

In [ ]:
previsores[1]

In [ ]:
# FORMA LOCAL -> previsão de um determinado registro
eli5.show_prediction(modelo, previsores[1], feature_names = list(credito)[0:20], show_feature_values=True)

In [ ]:
# SHAP -> técnica global que mostra a influência de cada atributo// a biblioteca atualizou e não está gerando o gráfico corretamente
X_treino2 = X_treino.astype(float)
explainer = shap.Explainer(modelo, X_treino2)
shap_values = explainer.shap_values(X_teste)
shap.summary_plot(shap_values, feature_names=list(credito)[0].values, plot_type='bar')
shap.force_plot(explainer.expected_value[1], shap_values[1])
shap.initjs()

In [ ]:
# INTERPRET
set_visualize_provider(InlineProvider())
ebm = ExplainableBoostingClassifier(feature_names=list(credito)[0:20])
ebm.fit(X_treino, y_treino)
global_explanation = ebm.explain_global()
show(global_explanation)